In [1]:
from bitarray import bitarray
from bitarray.util import ba2int, int2ba
import json

In [2]:
def load_sbox(file_path="sbox_gost_28147_89.json"):
    with open(file_path, "r") as file:
        return json.load(file)["S_BOX"]

In [3]:
messages = 'VUONG_DUY'
key = '136205V'
S_BOX = load_sbox()

In [4]:
S_BOX

[[12, 4, 6, 2, 10, 5, 11, 9, 14, 8, 13, 7, 0, 3, 15, 1],
 [6, 8, 2, 3, 9, 10, 5, 12, 1, 14, 4, 7, 11, 13, 0, 15],
 [11, 3, 5, 8, 2, 15, 10, 13, 14, 1, 7, 4, 12, 9, 6, 0],
 [12, 8, 2, 1, 13, 4, 15, 6, 7, 0, 10, 5, 3, 14, 9, 11],
 [7, 15, 5, 10, 8, 1, 6, 13, 0, 9, 3, 14, 11, 4, 2, 12],
 [5, 13, 15, 6, 9, 2, 12, 10, 11, 7, 8, 1, 4, 3, 14, 0],
 [8, 14, 2, 5, 6, 9, 1, 12, 15, 4, 11, 0, 13, 10, 3, 7],
 [1, 7, 14, 13, 0, 5, 8, 3, 4, 15, 10, 6, 9, 12, 11, 2]]

In [5]:
def hex_to_bin(hex_string):
    hex_string = hex_string.replace(" ", "")
    return bitarray(bin(int(hex_string, 16))[2:].zfill(len(hex_string) * 4))

In [6]:
def bin_to_hex(bin_array):
    """Chuyển đổi dữ liệu nhị phân thành chuỗi thập lục phân với định dạng mong muốn."""
    hex_string = format(ba2int(bin_array), 'X').zfill(len(bin_array) // 4)
    return ' '.join(hex_string[i:i+2] for i in range(0, len(hex_string), 2))

In [7]:
def str_to_block_64_bit(string):
    binary_string = bitarray()
    binary_string.frombytes(string.encode("ISO-8859-1"))
    padding = 64 - len(binary_string) % 64
    binary_string += bitarray(padding * "0")
    return [binary_string[i:i+64] for i in range(0, len(binary_string), 64)]

In [8]:
def block_64_bit_to_str(blocks):
    binary_string = bitarray()
    for block in blocks:
        binary_string += block
    return binary_string.tobytes().decode("ISO-8859-1")

In [9]:
def str_to_key_256_bit(string):
    binary_string = bitarray()
    binary_string.frombytes(string.encode("ISO-8859-1"))
    # lặp lại key cho đến khi đủ 256 bit
    while len(binary_string) < 256:
        binary_string += binary_string
    return binary_string[:256]

In [10]:
blocks = str_to_block_64_bit(messages)
key_256b = str_to_key_256_bit(key)
print(bin_to_hex(blocks[0]))
print(bin_to_hex(key_256b))

56 55 4F 4E 47 5F 44 55
31 33 36 32 30 35 56 31 33 36 32 30 35 56 31 33 36 32 30 35 56 31 33 36 32 30 35 56 31 33 36 32


In [11]:
def generate_subkeys(key):
    subkeys = []
    for i in range(8):
        subkeys.append(key[i*32:(i+1)*32])
    return subkeys

In [12]:
generate_subkeys(key_256b)

[bitarray('00110001001100110011011000110010'),
 bitarray('00110000001101010101011000110001'),
 bitarray('00110011001101100011001000110000'),
 bitarray('00110101010101100011000100110011'),
 bitarray('00110110001100100011000000110101'),
 bitarray('01010110001100010011001100110110'),
 bitarray('00110010001100000011010101010110'),
 bitarray('00110001001100110011011000110010')]

In [13]:
# phân chia các 8 khóa cho 32 vòng mã hóa
def generate_key_for_rounds(key):
    subkeys = generate_subkeys(key)
    keys_for_rounds = []
    for i in range(24):
        keys_for_rounds.append(subkeys[i % 8])
    for i in range(24, 32):
        keys_for_rounds.append(subkeys[7 - i % 8])
    return keys_for_rounds

In [14]:
generate_key_for_rounds(key_256b)

[bitarray('00110001001100110011011000110010'),
 bitarray('00110000001101010101011000110001'),
 bitarray('00110011001101100011001000110000'),
 bitarray('00110101010101100011000100110011'),
 bitarray('00110110001100100011000000110101'),
 bitarray('01010110001100010011001100110110'),
 bitarray('00110010001100000011010101010110'),
 bitarray('00110001001100110011011000110010'),
 bitarray('00110001001100110011011000110010'),
 bitarray('00110000001101010101011000110001'),
 bitarray('00110011001101100011001000110000'),
 bitarray('00110101010101100011000100110011'),
 bitarray('00110110001100100011000000110101'),
 bitarray('01010110001100010011001100110110'),
 bitarray('00110010001100000011010101010110'),
 bitarray('00110001001100110011011000110010'),
 bitarray('00110001001100110011011000110010'),
 bitarray('00110000001101010101011000110001'),
 bitarray('00110011001101100011001000110000'),
 bitarray('00110101010101100011000100110011'),
 bitarray('00110110001100100011000000110101'),
 bitarray('01

In [15]:
def apply_sbox(block, sbox):
    for i in range(8):
        index = ba2int(block[i*4:(i+1)*4])
        block[i*4:(i+1)*4] = int2ba(sbox[7-i][index], length=4)
    return block

In [16]:
# dịch bit sang trái 11 bit
def shift_11_bits_left(block):
    return block[11:] + block[:11]

In [17]:
def bitwise_addition_32bit(right_ba, subkey_ba):
    """
    Thực hiện phép cộng nhị phân modulo 2^32 giữa hai bitarray 32-bit.
    """
    right_int = ba2int(right_ba)
    key_round_int = ba2int(subkey_ba)
    print(f"Right: {right_ba.to01()}")
    print(f"Key: {subkey_ba.to01()}")

    # Cộng số nguyên và lấy modulo 2^32
    result_int = (right_int + key_round_int) % (2**32)

    # Chuyển đổi lại thành bitarray 32-bit
    result_ba = int2ba(result_int, length=32)
    print(f"Result: {result_ba.to01()}")

    return result_ba

In [18]:
def f_function(right, key_round):
    right = bitwise_addition_32bit(right, key_round)
    print(f"Right after addition: {bin_to_hex(right)}")
    right = apply_sbox(right, S_BOX)
    print(f"Right after S-Box: {bin_to_hex(right)}")
    right = shift_11_bits_left(right)
    print(f"Right after shift: {bin_to_hex(right)}")
    return right

In [19]:
def MAGMA_round(left, right, key_round):
    next_left = right
    next_right = left ^ f_function(right, key_round)
    return next_left, next_right

In [20]:
def MAGMA_cipher_block(block, key_for_rounds, encrypt=True):
    left, right = block[:32], block[32:]
    key_for_rounds = key_for_rounds if encrypt else key_for_rounds[::-1]
    
    results = []
    for key_round in key_for_rounds:
        left, right = MAGMA_round(left, right, key_round)
        results.append((left.copy(), right.copy(), key_round.copy()))
        
    final_result = right + left
    return final_result, results

In [21]:
def MAGMA_cipher(messages, key, encrypt=True):
    blocks = str_to_block_64_bit(messages)
    key_256b = str_to_key_256_bit(key)
    key_for_rounds = generate_key_for_rounds(key_256b)
    res_blocks = []
    for block in blocks:
        res_block, _ = MAGMA_cipher_block(block, key_for_rounds, encrypt)
        res_blocks.append(res_block)
    result = block_64_bit_to_str(res_blocks)
    return result

In [22]:
cipher_text = MAGMA_cipher(messages, key, encrypt=True)
print(cipher_text)
plain_text = MAGMA_cipher(cipher_text, key, encrypt=False)
print(plain_text)

Right: 01000111010111110100010001010101
Key: 00110001001100110011011000110010
Result: 01111000100100100111101010000111
Right after addition: 78 92 7A 87
Right after S-Box: 3F 75 67 19
Right after shift: AB 38 C9 FB
Right: 11111101011011011000011010110101
Key: 00110000001101010101011000110001
Result: 00101101101000101101110011100110
Right after addition: 2D A2 DC E6
Right after S-Box: EA 85 EC 0B
Right after shift: 2F 60 5F 54
Right: 01101000001111110001101100000001
Key: 00110011001101100011001000110000
Result: 10011011011101010100110100110001
Right after addition: 9B 75 4D 31
Right after S-Box: F0 A1 D9 34
Right after shift: 0E C9 A7 85
Right: 11110011101001000010000100110000
Key: 00110101010101100011000100110011
Result: 00101000111110100101001001100011
Right after addition: 28 FA 52 63
Right after S-Box: EF 03 45 52
Right after shift: 1A 2A 97 78
Right: 01110010000101011000110001111001
Key: 00110110001100100011000000110101
Result: 10101000010001111011110010101110
Right after addition:

In [23]:
# result for block 1
block = blocks[0]
key_for_rounds = generate_key_for_rounds(key_256b)

left, right, key_round = block[:32], block[32:], key_for_rounds[0]
print("Round 0:")
print(f"Left: {bin_to_hex(left)}")
print(f"Right: {bin_to_hex(right)}")
print(f"Key: {bin_to_hex(key_round)}")
print()
left, right = MAGMA_round(left, right, key_round)
print("Round 1:")
print(f"Left: {bin_to_hex(left)}")
print(f"Right: {bin_to_hex(right)}")
print(f"Key: {bin_to_hex(key_round)}")
print()

Round 0:
Left: 56 55 4F 4E
Right: 47 5F 44 55
Key: 31 33 36 32

Right: 01000111010111110100010001010101
Key: 00110001001100110011011000110010
Result: 01111000100100100111101010000111
Right after addition: 78 92 7A 87
Right after S-Box: 3F 75 67 19
Right after shift: AB 38 C9 FB
Round 1:
Left: 47 5F 44 55
Right: FD 6D 86 B5
Key: 31 33 36 32

